In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import importlib
from pathlib import Path

MY_SRC = '../src/'
if MY_SRC not in sys.path:
    sys.path.append(MY_SRC)

import text_model
import data

In [2]:
importlib.reload(data)

DATA_DIR = Path('../data/tfrecord')
TRAIN_DATA_PATH = DATA_DIR / 'training_data.tfrecord'
VALIDATION_DATA_PATH = DATA_DIR / 'validation_data.tfrecord'
TEST_DATA_PATH = DATA_DIR / 'testing_data.tfrecord'

training_data = data.make_dataset(str(TRAIN_DATA_PATH))
validation_data = data.make_dataset(str(VALIDATION_DATA_PATH))

In [6]:
test_data = data.make_dataset(str(TEST_DATA_PATH))

In [3]:
# Sanity check on the data

iterator = validation_data.make_one_shot_iterator()
with tf.Session() as sess:
    batch = sess.run(iterator.get_next())

print("".join(chr(t) for t in batch[0]['tokens'][0] if t))
print(batch[1][0])
plt.imshow(batch[0]['image'][0])

Leshrac's Rite
[0. 0. 1. 0.]


KeyError: 'image'

In [3]:
importlib.reload(text_model)

config = text_model.Config(lstm_layers=1,
                           lstm_size=256,
                           embedding_size=64,
                           attention_num_heads=3,
                           attention_head_size=64,
                           # embedding_regularization_coef=1e-4,
                           # dense_regularization_coef=1e-4,
                           feature_params={'vocab_size': 255},)
model = text_model.Model(config)
model.compile(optimizer=tf.keras.optimizers.Adam(clipnorm=5.0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
model.fit(training_data,
          validation_data=validation_data,
          epochs=40,
          # metrics=['accuracy', 'val_acc']
          steps_per_epoch=111,
          validation_steps=11)

/home/derick/.conda/envs/tf-1.12/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40


  1/111 [..............................] - ETA: 2:57 - loss: 1.3994 - acc: 0.1484

  2/111 [..............................] - ETA: 1:42 - loss: 1.3967 - acc: 0.3320

  3/111 [..............................] - ETA: 1:16 - loss: 1.3938 - acc: 0.4036

...

107/111 [===========================>..] - ETA: 1s - loss: 0.0438 - acc: 0.9920

108/111 [============================>.] - ETA: 0s - loss: 0.0442 - acc: 0.9918

109/111 [============================>.] - ETA: 0s - loss: 0.0441 - acc: 0.9918

110/111 [============================>.] - ETA: 0s - loss: 0.0440 - acc: 0.9918

111/111 [==============================] - 30s 271ms/step - loss: 0.0439 - acc: 0.9918 - val_loss: 0.0369 - val_acc: 0.9964


In [5]:
model.save_weights('./checkpoints/first-good-attention')


Consider using a TensorFlow optimizer from `tf.train`.


In [8]:
model.evaluate(test_data, steps=11)

 1/11 [=>............................] - ETA: 0s

 2/11 [====>.........................] - ETA: 0s

 3/11 [=======>......................] - ETA: 0s

 4/11 [=========>....................] - ETA: 0s

 5/11 [============>.................] - ETA: 0s

...

 8/11 [====================>.........] - ETA: 0s

 9/11 [=======================>......] - ETA: 0s

10/11 [==========================>...] - ETA: 0s

11/11 [==============================] - 1s 74ms/step


[0.04614141092381694, 0.9950284090909091]

In [9]:
importlib.reload(text_model)

config = text_model.Config(lstm_layers=1,
                           lstm_size=256,
                           embedding_size=64,
                           attention_num_heads=3,
                           attention_head_size=64,
                           # embedding_regularization_coef=1e-4,
                           # dense_regularization_coef=1e-4,
                           feature_params={'vocab_size': 255},)
model = text_model.SimpleModel(config)
model.compile(optimizer=tf.keras.optimizers.Adam(clipnorm=5.0),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_data,
          validation_data=validation_data,
          epochs=40,
          # metrics=['accuracy', 'val_acc']
          steps_per_epoch=111,
          validation_steps=11)

/home/derick/.conda/envs/tf-1.12/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40


  1/111 [..............................] - ETA: 2:33 - loss: 1.3821 - acc: 0.5078

  2/111 [..............................] - ETA: 1:28 - loss: 1.3747 - acc: 0.5000

  3/111 [..............................] - ETA: 1:07 - loss: 1.3626 - acc: 0.5130

...

107/111 [===========================>..] - ETA: 0s - loss: 0.0279 - acc: 0.9930

108/111 [============================>.] - ETA: 0s - loss: 0.0278 - acc: 0.9930

109/111 [============================>.] - ETA: 0s - loss: 0.0277 - acc: 0.9930

110/111 [============================>.] - ETA: 0s - loss: 0.0277 - acc: 0.9931

111/111 [==============================] - 28s 254ms/step - loss: 0.0276 - acc: 0.9932 - val_loss: 0.0265 - val_acc: 0.9950


In [ ]:
model.save_weights('./checkpoints/first-good-simple')


Consider using a TensorFlow optimizer from `tf.train`.


In [ ]:
model.evaluate(test_data, steps=11)